In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd


import os
import json


file_path = '/content/drive/My Drive/Dissertation/Europe/covid_subs.csv'
# Load the CSV files into DataFrames
covid_df = pd.read_csv(file_path)


ru_war_df = pd.read_csv('/content/drive/My Drive/Dissertation/Europe/russia-ukraine-subs.csv')

In [5]:
cleaned_df = ru_war_df[ru_war_df['author'] != '[deleted]']

cleaned_df

,author,id,created_utc,score,subreddit,subreddit_id,title,url,num_comments,score.1,selftext,timestamps
0,Porodicnostablo,rt70v0,1.640996e+09,297,europe,t5_2qh4j,Happy New Year from Belgrade,https://i.redd.it/s54eet1ryy881.jpg,10,297,NaN,2022-01-01 00:17:20
1,sir_captaindoge,rt75at,1.640997e+09,13,europe,t5_2qh4j,Happy New Year from Portugal!,https://www.reddit.com/r/europe/comments/rt75a...,1,13,Happy New Year to all countries in Europe! I w...,2022-01-01 00:23:39
2,gsurfer04,rt7az4,1.640997e+09,13,europe,t5_2qh4j,Happy New Year for all Europe!,https://www.reddit.com/r/europe/comments/rt7az...,0,13,"With midnight passing in UTC+0, may 2022 be a ...",2022-01-01 00:32:17
5,BlinkVideoEdits,rt7hjy,1.640998e+09,74,europe,t5_2qh4j,Fireworks across London,https://v.redd.it/wperybwa3z881,2,74,NaN,2022-01-01 00:42:34
6,pgarson,rt7nyz,1.640998e+09,0,europe,t5_2qh4j,Spain says it is first in Europe to officially...,https://www.theguardian.com/world/2021/dec/31/...,37,0,NaN,2022-01-01 00:52:23
...,...,...,...,...,...,...,...,...,...,...,...,...
156759,SpecificBasil7850,18urzq6,1.703976e+09,2291,europe,t5_2qh4j,People's reaction when a foreigner tries speak...,https://i.redd.it/tc4mvwzvfi9c1.png,529,2291,NaN,2023-12-30 22:42:42
156760,uncl3mar1k,18usaj0,1.703977e+09,1,europe,t5_2qh4j,Key Moments of Ukraine War 2023,https://v.redd.it/flrsumd6ii9c1,1,1,NaN,2023-12-30 22:55:18
156761,Resident_Papaya8422,18uskk2,1.703978e+09,1,europe,t5_2qh4j,Hello👋,https://i.redd.it/a865k9idki9c1.jpeg,1,1,NaN,2023-12-30 23:07:21
156762,BeatSmithAI,18ut15x,1.703979e+09,1,europe,t5_2qh4j,History - Roman Firefighters [History Facts],https://www.youtube.com/watch?v=Dcfb5QqKRDU,0,1,NaN,2023-12-30 23:27:29


In [6]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download stopwords if not already available
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import re

# Assuming cleaned_df is your DataFrame
# Create a combined text column from 'title' and 'selftext'
cleaned_df['text'] = cleaned_df['title'].fillna('') + ' ' + cleaned_df['selftext'].fillna('')

cleaned_df.columns

<ipython-input-7-111f49b66646>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['text'] = cleaned_df['title'].fillna('') + ' ' + cleaned_df['selftext'].fillna('')


Index(['author', 'id', 'created_utc', 'score', 'subreddit', 'subreddit_id',
       'title', 'url', 'num_comments', 'score.1', 'selftext', 'timestamps',
       'text'],
      dtype='object')

In [8]:
pip install sentence-transformers


In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Assuming 'cleaned_df' is your DataFrame and it has a column 'text' with subreddit posts

# Debugging step: Print DataFrame structure
print("DataFrame structure:")
print(cleaned_df.head())

# Ensure the DataFrame contains the 'text' column
if 'text' not in cleaned_df.columns:
    raise ValueError("The DataFrame does not contain a 'text' column.")

# Load the pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example dataset of subreddit posts
posts = cleaned_df['text'].tolist()

# Encode the posts into embeddings
print("Encoding posts...")
post_embeddings = model.encode(posts, show_progress_bar=True)



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


KeyboardInterrupt: 

DataFrame structure:
            author      id   created_utc  score subreddit subreddit_id  \
0  Porodicnostablo  rt70v0  1.640996e+09    297    europe     t5_2qh4j   
1  sir_captaindoge  rt75at  1.640997e+09     13    europe     t5_2qh4j   
2        gsurfer04  rt7az4  1.640997e+09     13    europe     t5_2qh4j   
5  BlinkVideoEdits  rt7hjy  1.640998e+09     74    europe     t5_2qh4j   
6          pgarson  rt7nyz  1.640998e+09      0    europe     t5_2qh4j   

                                               title  \
0                       Happy New Year from Belgrade   
1                      Happy New Year from Portugal!   
2                     Happy New Year for all Europe!   
5                            Fireworks across London   
6  Spain says it is first in Europe to officially...   

                                                 url  num_comments  score.1  \
0                https://i.redd.it/s54eet1ryy881.jpg            10      297   
1  https://www.reddit.com/r/europe/comm

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Encoding posts...


Batches:   0%|          | 0/3933 [00:00<?, ?it/s]

In [10]:
# Define a query related to climate change
query = "russia ukraine war"

# Encode the query into an embedding
print("Encoding query...")
query_embedding = model.encode(query)

# Compute cosine similarity between the query embedding and post embeddings
print("Computing similarity scores...")
similarity_scores = cosine_similarity([query_embedding], post_embeddings)[0]

# Define a similarity threshold
threshold = 0.7

# Filter posts based on the similarity threshold
print("Filtering posts...")
filtered_posts = [post for post, score in zip(posts, similarity_scores) if score > threshold]


# Optional: If you want to save the filtered posts to a new DataFrame
filtered_df = cleaned_df.loc[cleaned_df['text'].isin(filtered_posts)]
filtered_df


Encoding query...
Computing similarity scores...
Filtering posts...


,author,id,created_utc,score,subreddit,subreddit_id,title,url,num_comments,score.1,selftext,timestamps,text
1692,Transeuropeanian,rytffk,1.641623e+09,3231,europe,t5_2qh4j,How Russia’s Military Is Positioned to Threate...,https://i.redd.it/ihvbeb9lrea81.jpg,859,3231,NaN,2022-01-08 06:29:29,How Russia’s Military Is Positioned to Threate...
2504,Korefanz,s18tiu,1.641892e+09,326,europe,t5_2qh4j,"State Dep: Russia Has Invaded, Occupied, Fomen...",https://georgianjournal.ge/politics/37664-stat...,195,326,NaN,2022-01-11 09:11:29,"State Dep: Russia Has Invaded, Occupied, Fomen..."
2762,Ciaran123C,s240rp,1.641987e+09,42,europe,t5_2qh4j,"State Dep: Russia Has Invaded, Occupied, Fomen...",https://georgianjournal.ge/politics/37664-stat...,3,42,NaN,2022-01-12 11:21:42,"State Dep: Russia Has Invaded, Occupied, Fomen..."
4785,Ivan_Matrosoff,s7ya0o,1.642621e+09,1,europe,t5_2qh4j,Why Ukraine is invading Russia?,https://www.reddit.com/r/europe/comments/s7ya0...,0,1,[removed],2022-01-19 19:30:57,Why Ukraine is invading Russia? [removed]
5117,molokoplus359,s8rkav,1.642709e+09,5,europe,t5_2qh4j,Ukraine war necessary if Russia recognises bre...,https://www.reuters.com/world/europe/ukraine-w...,1,5,NaN,2022-01-20 19:59:36,Ukraine war necessary if Russia recognises bre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145947,SpecialistMotor3677,17g47bx,1.698239e+09,1,europe,t5_2qh4j,Russian Invasion of Ukraine during 1⅔ Years: E...,https://youtu.be/T9Mw80xPmY4?si=Sl8YEx8f_oCRx4Jb,0,1,NaN,2023-10-25 13:00:05,Russian Invasion of Ukraine during 1⅔ Years: E...
151974,BalticsFox,1896ldn,1.701532e+09,14,europe,t5_2qh4j,Scientists in Russia struggle in a world trans...,https://www.science.org/content/article/scient...,3,14,NaN,2023-12-02 15:54:45,Scientists in Russia struggle in a world trans...
152165,WitteringLaconic,18a49en,1.701640e+09,1,europe,t5_2qh4j,Ukraine war: Russia accused of killing unarmed...,https://www.bbc.co.uk/news/world-europe-67607857,1,1,NaN,2023-12-03 21:44:34,Ukraine war: Russia accused of killing unarmed...
153245,Grouchy_Letter_6077,18eiqgg,1.702144e+09,1,europe,t5_2qh4j,Ukraine/Russia war,https://i.redd.it/zhed74q43b5c1.jpg,0,1,[removed],2023-12-09 17:41:42,Ukraine/Russia war [removed]


In [11]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install bitsandbytes


In [13]:
# Load model directly
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map='auto', torch_dtype=torch.float16)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [16]:
filtered_df.columns

Index(['author', 'id', 'created_utc', 'score', 'subreddit', 'subreddit_id',
       'title', 'url', 'num_comments', 'score.1', 'selftext', 'timestamps',
       'text'],
      dtype='object')

In [18]:
# Sort the dataframe by the 'num_comments' column in descending order
df_sorted = filtered_df.sort_values(by='num_comments', ascending=False)

# Display the top 10 posts with the most comments
df_sorted = df_sorted.head(15)
print(top_posts[['author', 'id', 'created_utc', 'score', 'subreddit', 'subreddit_id',
       'title', 'url', 'num_comments', 'score.1', 'selftext', 'timestamps',
       'text']])


                      author       id   created_utc  score subreddit  \
22095         PanEuropeanism   tb9a4r  1.646948e+09   2702    europe   
1692        Transeuropeanian   rytffk  1.641623e+09   3231    europe   
124184        Beyond_The_Dim  14qa3vh  1.688462e+09    109    europe   
20738            BestButtons   t77924  1.646478e+09   2870    europe   
90341   Discipline_Accountan  10d7n5v  1.673852e+09    322    europe   
2504                Korefanz   s18tiu  1.641892e+09    326    europe   
19146      ARoyaleWithCheese   t3ismy  1.646064e+09    393    europe   
29955         TriggurWarning   ub3d4n  1.650832e+09    417    europe   
19932              MorgrainX   t5agh0  1.646256e+09    716    europe   
18919          TheRealMykola   t33meo  1.646013e+09    800    europe   
23432        Nameless_User07   tgif7k  1.647546e+09    427    europe   
28891     DoppelgangerHornet   u68l1h  1.650272e+09    585    europe   
36166               snooshoe   uyhzyq  1.653600e+09    442    eu

In [49]:
def analyze_sentiment(text):
    # Create a prompt that asks for a direct sentiment classification
    prompt = f"What is the sentiment of this text? '{text}' Please answer with only 'positive', 'negative', or 'neutral'."

    # Tokenize the input text
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate the model's response
    outputs = model.generate(**inputs, max_length=100, do_sample=True)

    # Decode the response to get the sentiment
    sentiment = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process the sentiment to only allow specific categories
    sentiment = sentiment.lower().strip()
    return sentiment  # Default to neutral if the response is unclear



# Apply the sentiment analysis function to each post
top_posts['sentiment'] = top_posts['text'].apply(analyze_sentiment)

# Display the results
print(top_posts[['text', 'sentiment']].head(10))



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

                                                     text  \
22095   Not just Putin: Most Russians support the war ...   
1692    How Russia’s Military Is Positioned to Threate...   
124184  Why is every Russian responsible for the war? ...   
20738   War in Ukraine: The Russians leaving Russia fo...   
90341   Croatian president: USA and Russia are 'waging...   
2504    State Dep: Russia Has Invaded, Occupied, Fomen...   
19146   Analysis: Ukraine war tests growing China-Russ...   
29955   Russian state TV talks of "War against Europe ...   
19932   Russia’s secret documents: war in Ukraine was ...   
18919   Sources: Belarus to join Russia’s war on Ukrai...   

                                                sentiment  
22095   what is the sentiment of this text? 'not just ...  
1692    what is the sentiment of this text? 'how russi...  
124184  what is the sentiment of this text? 'why is ev...  
20738   what is the sentiment of this text? 'war in uk...  
90341   what is the sentimen

<ipython-input-49-ec8819619116>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_posts['sentiment'] = top_posts['text'].apply(analyze_sentiment)


In [60]:
first_post_text = top_posts.iloc[5]['sentiment']
first_post_text

"what is the sentiment of this text? 'state dep: russia has invaded, occupied, fomented conflict against ukraine, georgia, moldova'please answer with only 'positive', 'negative', or 'neutral'. \n\nthis text is 'negative'. \nthe text is an official statement from the state department, which is a government agency, expressing its official stance on russia's actions in ukraine, georgia, and moldova. the language used is formal and objective, but the content"